In [1]:
import cv2
from keras_facenet import FaceNet
import numpy as np

In [2]:
model = FaceNet()

In [4]:
database = {}

In [5]:
def register_person(person_id, image):
    resized_image=preprocess_image(image)
    embedding = model.embeddings([resized_image])[0]
    database[person_id] = embedding
    print(f"Person {person_id} registered successfully.")

In [6]:
def capture_image():
    cap = cv2.VideoCapture(0)
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture image")
        return None
    cap.release()
    return frame

In [7]:
def preprocess_image(image):
    resized_image = cv2.resize(image, (160, 160))
    cv2.imshow('Image', resized_image)

    # Wait for a key press and close the window
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return resized_image

In [8]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [9]:
def find_match(new_embedding, database, threshold):
    distances = []
    person_ids = []
    for person_id, stored_embedding in database.items():
        distance = np.linalg.norm(new_embedding - stored_embedding)
        distances.append(distance)
        person_ids.append(person_id)
    print(distances)
    if min(distances) > threshold:
        return None,min(distances)
    distances = np.array(distances)
    probabilities = softmax(-distances)
    
    # Get the person with the highest probability
    max_index = np.argmax(probabilities)
    person = person_ids[max_index]
    max_probability = probabilities[max_index]
    return person, max_probability

In [10]:
def recognize_person(new_image, threshold=0.8):
    resized_image = preprocess_image(new_image)
    new_embedding = model.embeddings([resized_image])[0]
    person_id, distance = find_match(new_embedding, database, threshold)
    if person_id is not None:
        print(f"Match found: {person_id} with distance {distance}")
    else:
        print(f"No match found distance {distance} {threshold}")

In [11]:
def recognize():
    print("Capturing image for recognition...")
    new_image = capture_image()
    if new_image is not None:
        image=preprocess_image(new_image)
        recognize_person(new_image,0.6)
    else:
        print("No image captured for recognition")

In [12]:
def register(id:str):
    person_id = id
    image = capture_image()
    cv2.imshow('Image', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    if image is not None:
        image = preprocess_image(image)
        register_person(person_id, image)
    else:
        print("No image captured for registration")

In [13]:
id:int = 1

In [ ]:
print("Face recognition project")
print("1.register")
print("2.recognize")
choice=int(input("Enter you choice:"))
if choice==1:
    s=input("Enter Name:")
    register(s)
    id+=1
elif choice==2:
    recognize()

Face recognition project
1.register
2.recognize


Enter you choice: 1
Enter Name: hamdan
